This cell will import all of the necessary libraries for this lab. These 
include the pyserial library which is used to communicate with the
microcontroller, NumPy which is used to load data from a CSV file, and
pyplot from matplotlib which is a plotting package with almost identical
commands to MATLAB. Run this cell first.

In [9]:
from serial import Serial
import time
import numpy as np
from matplotlib import pyplot as plt

This cell defines a function which will plot the step response. No need to
modify this cell.

In [10]:
def plot_step(kp, ki, sp, filename):
    ''' Plot the step response

    Gives a subplot for the velocity vs time, and for the controller outputs
    vs time. Also places the gains on the plot title.

    Args:
        kp:     Proportional Gain
        ki:     Integral Gain
        sp:     Velocity setpoint to step to
    '''
    # Import the data from the file
    data = np.genfromtxt(filename, delimiter=',')
    times = data[:,0]
    velocities = data[:,1]
    percents = data[:,2]
    p_percents = data[:,3]
    i_percents = data[:,4]

    # Make a figure and set the size to 6x6 in
    fig = plt.figure()
    plt.rcParams['figure.figsize'] = (6, 6)

    # Plot the step response of position vs time on the top half
    plt.subplot(211)
    plt.plot(times, velocities)
    plt.hlines(sp, times[1], times[-1], colors='k', linestyles='dashed')
    plt.xlabel("Time [s]")
    plt.ylabel("Velocity [rad/s]")
    plt.title(f'Step Response: K_P = {kp}, K_I = {ki}')

    # Plot the proportional and integral components on the bottom half
    plt.subplot(212)
    plt.plot(times, p_percents, '--')
    plt.plot(times, i_percents, '--')
    plt.plot(times, percents)
    plt.ylim(-110, 110)
    plt.xlabel("Time [s]")
    plt.ylabel("Voltage percent")
    plt.title("Controller Output")
    plt.legend(['P Part', 'I Part', 'Total'])
    plt.tight_layout()
    plt.show()

This cell defines a function which will connect to the microcontroller over
the serial port. It will send the desired gains, velocity setpoint to step to,
and total time to run the test for, as specified in the variables. Then it will
collect the data and write it to `filename`. Finally it will plot the step
response using the previous function.
No need to modify this cell.

In [15]:
def run_step(kp, ki, sp, t_step, filename, out_max=100):
    # Uncomment this line and give the name of the serial port which the Nucleo
    # is connected to, as a string. This should be something like
    # 'COMx' for Windows and
    # '/dev/tty.usbmodemXXXX' for Mac.
    # Windows: Open Device Manager and look under "Ports (COM & LPT)" for a 
    # "USB Serial Device"
    # Mac/Linux: Open terminal and run "ls /dev/tty.*" to find the port matching the
    # format above.
    serport = 'COM16'
    
    # Open the serial port, and give it the name ser.
    with Serial(serport, baudrate=115200, timeout=5) as ser:
        # Restart the microcontroller
        ser.write(b'\x02\x04')
        # Write the parameters to the microcontroller
        ser.write((str(kp) + '\r\n').encode())
        ser.write((str(ki) + '\r\n').encode())
        ser.write((str(sp) + '\r\n').encode())
        ser.write((str(out_max) + '\r\n').encode())
        ser.write((str(t_step) + '\r\n').encode())

        # Open the data file for writing, and give it the name datafile
        with open(filename, 'w') as datafile:
            while True:
                # Read the current line and decode from bytes to a string
                line = ser.readline().decode()
                # Print the line so we can see what the microcontroller is doing
                # print(line, end='')
                # If the line starts with END, then we know the data is complete
                if line[:3] == 'END':
                    break
                # If the line has a comma, it is a line of CSV data.
                elif ',' in line:
                    datafile.write(line.strip() + '\n')

    plot_step(kp, ki, sp, filename)

Make several cells here to call `run_step` for each of the required gains
from the lab manual. Use a unique filename for the data files. An example is
below which runs a step response with $K_P$ = 0.1 [%/(rad/s)], $K_I$ = 0, the
velocity to step to of 500 rad/s, total test time of 1000 ms (1 second), and
filename to save data to of "data1.csv".

In [ ]:
run_step(0.1, 0, 500, 1000, "data1.csv")

Add more cells below using the Cell menu, or by pressing the 'b' key while in
cell mode (press escape to enter cell mode) and highlighting the bottom cell.
Then add one call of `run_step` per cell for the different gains specified in
the lab manual. Make the test time long enough to see the systems get to
steady state.